In [ ]:
import matplotlib.patches as mpatches
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 10,
   'legend.fontsize': 10,
   'xtick.labelsize': 10,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [6.5, 2] # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## helper function

In [ ]:
def autolabel(rects, ses):
    # attach some text labels
    for rect, se in zip(rects, ses):
        height = rect.get_height()
        plt.axes().text(rect.get_x()+rect.get_width()/2., height+1.03*se, '{:2.3f}'.format(float(height)).lstrip('0'),
                ha='center', va='bottom',fontsize=10)

## Fixed Parameter

In [ ]:
# experiment
exp = 'imgpred'
exp_label = 'Scene Recall Matching'

# dictionaries
chance_lvl = 1/50 # chance level, 34.5 scenes per subjects on average
feat_dict = {'multi_srm':[125],'multi_dict':[100],'indv_srm':[125],'indv_ica':[25],'indv_dict':[100], 'indv_gica':[25],'avg':[50]}
roi_dict = {'pmc':0}
md_dict = {'avg':'MNI','multi_srm':'MDMS','ica':'ICA','indv_gica':'GICA','indv_dict':'DL','indv_srm':'SRM','multi_dict':'MDDL'}
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}
lg_dc = ['Prm.','Prm.+Sec.']

# path
input_file = '../../output/accu_bar/{}/{}/{}/{}.npz' #exp,total_ds,roi,model
output_path = '../../output/figures/{}/' #exp,total_ds
output_file = output_path+'{}' #roi

## Plotting Parameter

In [ ]:
total_ds = list(range(12))
max_accu = 0.17
datasets = [3]
model_all = ['avg','indv_gica','indv_dict','indv_srm','multi_dict','multi_srm']
roi_all = ['pmc']
if not os.path.exists(output_path.format(exp).replace(' ','')):
    os.makedirs(output_path.format(exp).replace(' ',''))

## Aggregate Accuracies

In [ ]:
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    for model in model_all: 
        ws=np.load(input_file.format(exp,total_ds,roi,model).replace(' ',''))
        roi_mean.append(ws['mean'].item())
        roi_se.append(ws['se'].item())    
    all_mean.append(roi_mean)
    all_se.append(roi_se)

## Plot all roi

In [ ]:
width=1
nmodel = len(model_all)# number of models
ndata = len(datasets)
group_width = 1.1*width*nmodel+0.4*width
center_all = np.linspace(0,group_width*(ndata-1),ndata)
# set colors
color_all = ['lightgrey','mediumseagreen','dodgerblue','mediumorchid','gold','red']
pattern = ['///',' ']

for m,roi in enumerate(roi_all):
    # xtick 
    xtick_idx = []
    xtick_name = []  
    xtick_idx.append(center_all[0]-0.5*width-0.5)
    xtick_name.append('model\nk')
    # dataset names
    ds_idx = []
    ds_name = []
    plt.figure()
    # configure bars
    for i,(ds, center) in enumerate(zip(datasets,center_all)):
        mean = list(all_mean[m][i*nmodel:(i+1)*nmodel])
        se   = list(all_se[m][i*nmodel:(i+1)*nmodel])    
        idx = np.concatenate(((center-0.15*width)[None],np.arange(center+width,center+nmodel*width-2.2*width,width),\
                              (center+nmodel*width-1.85*width)[None],(center+nmodel*width-0.85*width)[None]))
        error_config = {'ecolor': '0','capsize':3}   
        rects = plt.bar(idx, mean, yerr=se, align='center', error_kw=error_config, width = width-0.1)
        # set colors
        for rect_i in range(len(rects)):
            rects[rect_i].set_color(color_all[rect_i])
        # add patterns
        for rect_i in range(1,len(rects)-2):
            rects[rect_i].set_hatch(pattern[0])
            
        autolabel(rects,se)
        ds_idx.append(center+(nmodel-3.85)*width/2) 
        ds_name.append(ds_dict[ds])
        # xtick names
        xtick_idx.extend(idx)
        for model in model_all:
            feat = str(feat_dict[model][roi_dict[roi]])
            if model == 'avg':
                xtick_name.append(md_dict[model]+'\n ')
            else:
                xtick_name.append(md_dict[model]+'\n'+feat)

    plt.xticks(xtick_idx,xtick_name)
    plt.yticks(np.arange(0, max_accu+0.001, 0.1))
    plt.ylabel('Accuracy')
    left_lim = center_all[0]-0.5*width-0.5
    right_lim = center_all[-1]+(nmodel-0.5)*width+0.5
    plt.xlim([left_lim,right_lim])
    plt.ylim([0,max_accu])

    # plot chance accuracy
    line = plt.plot([center_all[0]-0.5*width-0.5, center_all[0]+(nmodel+1)*width], [chance_lvl, chance_lvl], 'k-.', linewidth=2)

    # Add texts
    plt.text((right_lim+left_lim)/2, 1.2*max_accu, exp_label+' (ROI: '+roi.upper()+')',fontsize=13,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    for d in range(len(datasets)):
        plt.text(ds_idx[d],0.77*max_accu,'Prm.: '+ds_name[d],bbox=props,fontsize=11,horizontalalignment='center', verticalalignment='top')

    # legend
    legend_handle = []
    for patch_i in range(len(model_all)):
        legend_handle.append(mpatches.Patch(color=color_all[patch_i], label=md_dict[model_all[patch_i]]))
    l1 = plt.legend(handles=legend_handle,bbox_to_anchor=(0.,1.02,1.,.102),loc=3,ncol=len(model_all),mode="expand", borderaxespad=0.)
    ax = plt.gca().add_artist(l1)
    l2 = plt.legend(line ,['chance'],bbox_to_anchor=(0.,1.04,0.82,0.),loc=1,ncol=1)
    ax = plt.gca().add_artist(l2)
    ax = plt.gca().add_artist(l1)
    legend_handle2 = []
    for patch_i in range(2):
        legend_handle2.append(mpatches.Patch(hatch=pattern[patch_i], color='beige',label=lg_dc[patch_i]))

    plt.legend(handles=legend_handle2,bbox_to_anchor=(-0.01,1.04,1.,0.),loc=2,ncol=2)        

#     plt.savefig(output_file.format(exp,roi).replace(' ','')+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,roi).replace(' ','')+'.pdf', format='pdf', dpi=200,bbox_inches='tight')